In [ ]:
!pip install dowhy gcastle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.4/398.4 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.7 MB/s eta 0:00:00
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.12
    Uninstalling Cython-3.0.12:
      Successfully uninstalled Cython-3.0.12


In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import statsmodels.api as sm
from dowhy import gcm
from dowhy import CausalModel

In [ ]:
# Sample Data
S = np.random.normal(loc=0.5, scale=1, size=2000)
X = 2*S + np.random.normal(loc=0, scale=0.5, size=2000)
Y = 3*S + 1.5*X + np.random.normal(loc=1,scale=0.5, size=2000)

# Interventional calculation

In [ ]:
# Estimate causal effect using back-door criterion
# S -> X -> Y
#  \-------^

data = pd.DataFrame({'X': X, 'S': S})
data = sm.add_constant(data, prepend=True)
model = sm.OLS(Y, data) # Ordinary Linear Regression
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                 1.502e+05
Date:                Wed, 19 Feb 2025   Prob (F-statistic):               0.00
Time:                        14:53:38   Log-Likelihood:                -1431.5
No. Observations:                2000   AIC:                             2869.
Df Residuals:                    1997   BIC:                             2886.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0088      0.012     81.278      0.000       0.984       1.033
X              1.4799      0.022     66.070      0.000       1.436       1.524
S              3.0438      0.046     65.926      0.000       2.953       3.134
==============================================================================
Omnibus:                        0.880   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.644   Jarque-Bera (JB):                0.816
Skew:                          -0.046   Prob(JB):                        0.665
Kurtosis:                       3.039   Cond. No.                         12.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# Calculate P(Y|do(X=1))

X1 = np.array([1.0]*len(X))
Y1 = results.params['X']*X1 + results.params['S']*S + results.params['const']
np.average(Y1)


4.0286552410407

In [ ]:
# Calculating Intervention using DoWhy GCM Model
# https://www.pywhy.org/dowhy/v0.12/user_guide/causal_tasks/what_if/interventions.html\
# Hint: you can also use gcm.auto.assign_causal_mechanisms (see: https://www.pywhy.org/dowhy/v0.10.1/user_guide/modeling_gcm/draw_samples.html)

# PROBLEM 1 IMPLEMENT INTERVENTION USING DoWHY GCM MODEL
# [ YOUR CODE HERE ]

Fitting causal mechanism of node S: 100%|██████████| 3/3 [00:00<00:00, 226.36it/s]


4.096401749043804

# Counterfactual calculation

In [ ]:
data_gcm2 = pd.DataFrame({'X': X, 'S': S, 'Y': Y})
causal_model2 = gcm.InvertibleStructuralCausalModel(nx.DiGraph([('X', 'Y'), ('S', 'X'), ('S', 'Y')]))
gcm.auto.assign_causal_mechanisms(causal_model2, data_gcm2)
gcm.fit(causal_model2, data_gcm2)

gcm.counterfactual_samples(
    causal_model2,
    {'X': lambda x: 1},
    observed_data=pd.DataFrame(data=dict(X=[0], Y=[2], S=[0.5])))

Fitting causal mechanism of node S: 100%|██████████| 3/3 [00:00<00:00, 127.68it/s]


,S,X,Y
0,0.5,1,3.454063


# Causal Discovery

In [ ]:
from castle.algorithms import PC, ICALiNGAM

In [ ]:
pc = PC()
pc_dataset = np.vstack([X, Y, S]).T
pc.learn(pc_dataset)

In [ ]:
pc.causal_matrix

# PROBLEM 2 - WHY IS THE CAUSAL METRIX ALMOST COMPLETE?
# CAN YOU USE REFUTION TEST TO CHECK FOR CAUSALITY DIRECTION?
# IMPLEMENT AND DISCUSS

Tensor([[0, 1, 1],
        [1, 0, 1],
        [1, 1, 0]])

In [ ]:
N = 2000
a = np.random.uniform(0, 1, N)
b = np.random.uniform(3, 6, N)
c = a + b + .1 * np.random.uniform(-2, 0, N)
d = .7 * c + .1 * np.random.uniform(0, 1, N)
lingam_dataset = np.vstack([a, b, c, d]).T

In [ ]:
lingam = ICALiNGAM(random_state=1)
lingam.learn(lingam_dataset)

In [ ]:
lingam.weight_causal_matrix

Tensor([[0.   , 0.   , 1.002, 0.   ],
        [0.   , 0.   , 1.001, 0.   ],
        [0.   , 0.   , 0.   , 0.7  ],
        [0.   , 0.   , 0.   , 0.   ]])